In [1]:
import numpy as np
import pydot
from IPython.display import HTML, SVG, display
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    GenerateHtml,
    InverseDynamicsController,
    PidController,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    Simulator,
    StartMeshcat,
    SceneGraph,
    PassThrough,
    Demultiplexer,
)

from catbot.utils.meshcat_util import MeshcatCatBotSliders 

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [11]:
time_step = 1e-3

builder = DiagramBuilder()

# -- Add original plant -- #
# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
# Note that we parse into both the plant and the scene_graph here.
model = Parser(plant, scene_graph).AddModelFromFile(
    "../models/singleAxisCatBot.urdf")

gravity_field = plant.mutable_gravity_field()
gravity_field.set_gravity_vector(np.array([0.0, 0.0, 0.0]))

plant.Finalize()

# print('plant names: ', plant.GetPositionNames(model))
# print('plant state: ', plant.GetState(model))

# -- Add controller plant -- #
controller_plant = MultibodyPlant(time_step=time_step)
model = Parser(controller_plant).AddModelFromFile(
    "../models/singleAxisCatBot.urdf")
controller_gravity_field = controller_plant.mutable_gravity_field()
controller_gravity_field.set_gravity_vector(np.array([0.0, 0.0, 0.0]))
controller_plant.Finalize()

# -- Add visualizer -- #
visualizer = MeshcatVisualizer.AddToBuilder(builder, 
                                            scene_graph.get_query_output_port(),
                                            meshcat)

meshcat.ResetRenderMode()
meshcat.DeleteAddedControls()

# -- Add controller -- #
# Create a PID controller for each joint.
num_model_actuators = controller_plant.num_actuators()

# state_projection_matrix = np.array([
#     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#     [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#     [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
#     [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],

#     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
#     [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
#     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
#     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
# ])

# actuator pos is a_rev, b_rev, a_hinge, b_hinge
# state is Center, a_hinge?, a_rot, b_hinge, b_rot 
# idk why they're different
state_projection_matrix = np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],

    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
])

# kp = [0.0000, 0.000, 0.02, 0.02]
# ki = [0.00, 0.00, 0.0, 0.0]
# kd = [0.00, 0.00, 0.02, 0.02]

kp = [0.03, 0.03, 0.03, 0.03]
ki = [0.00, 0.00, 0.0, 0.0]
kd = [0.025, 0.025, 0.03, 0.03]
catbot_controller = builder.AddSystem(
    PidController(state_projection_matrix, kp, ki, kd)
)

catbot_controller.set_name("catbot_controller")
builder.Connect(
    plant.get_state_output_port(model),
    catbot_controller.get_input_port_estimated_state(),
)
builder.Connect(
    catbot_controller.get_output_port_control(), 
    plant.get_actuation_input_port()
)

# -- Set up teleop wigits -- #


teleop = builder.AddSystem(
    MeshcatCatBotSliders(
        meshcat,
    ))

builder.Connect(teleop.get_output_port(0), catbot_controller.get_input_port_desired_state())
builder.Connect(plant.get_state_output_port(), teleop.get_input_port(0))

diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

simulator.set_target_realtime_rate(1.0)
meshcat.AddButton("Stop Simulation", "Escape")
print("Press Escape to stop the simulation")

# -- Set init pose -- #
# Center, b_hinge?, b_rot, a_hinge, a_rot
q0 = [0.0, 0.00, 0.00, 0.0, 0.0]
# q0 = np.array([0.0, np.pi/3, 0.0, 0.0, 0.0])
plant_context = plant.GetMyMutableContextFromRoot(context)
plant.SetPositions(plant_context, q0)

simulator.AdvanceTo(0.01)



# -- Uncomment to simulate -- #
cnt = 0
while meshcat.GetButtonClicks("Stop Simulation") < 1:
    simulator.AdvanceTo(simulator.get_context().get_time() + 1.0)
    # if cnt % 10 == 0:
    #     print_status()
    #     input('Continue')
    # cnt += 1

meshcat.DeleteButton("Stop Simulation")

Keyboard Controls:
a_rev : KeyQ / KeyE
b_rev : KeyW / KeyS
a_hinge : KeyA / KeyD
b_hinge : KeyJ / KeyL
Press Escape to stop the simulation
Pos:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
value:  [0.6, 0.0, 0.0, 0.0]
des state:  [0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[-0.22503872  0.          0.46578484  0.         -0.00070481 -0.07839523
  0.          0.16292163  0.         -0.00090483]
value:  [0.6, 0.7000000000000001, 0.0, 0.0]
des state:  [0.6, 0.7000000000000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[-0.61446093  0.          0.59470442  0.          0.6751815  -0.01749778
  0.          0.00610305  0.          0.03005903]
value:  [0.6, 0.7000000000000001, -0.48, 0.0]
des state:  [0.6, 0.7000000000000001, -0.48, 0.0, 0.0, 0.0, 0.0, 0.0]
[-0.62291433 -0.21011787  0.59793376  0.          0.69086257 -0.00431321
 -0.31674883  0.00235492  0.          0.01103339]
value:  [0.6, 0.7000000000000001, -0.48, -0.39]
des state:  [0.6, 0.7000000000000001, -0.48, -0.39, 0.0, 0.0, 0.